In [12]:
import os
import numpy as np
#import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
main_directory = '/content/drive/MyDrive/TESS_changed'

In [15]:
# Load data and extract features and labels
#data = pd.read_csv("your_data.csv")
#X = data.drop("emotion", axis=1).values
#y = data["emotion"].values

X= []
Y= []

In [16]:
# Mapping between folder names and labels
label_mapping = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'neutral': 4,
    'pleasant_suprise': 5,
    'sad': 6
}

In [17]:
# Iterate through each subdirectory (each emotion)
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)

    if os.path.isdir(folder_path):
        label = label_mapping.get(folder_name, -1)  # -1 if folder name not found in the mapping
        if label != -1:
            # Iterate through each .wav file in the subdirectory
            for filename in os.listdir(folder_path):
                if filename.endswith(".wav"):
                    # Load audio file using librosa
                    audio_path = os.path.join(folder_path, filename)
                    audio, _ = librosa.load(audio_path, sr=None)  # sr=None to preserve the original sampling rate

                    # Extract Mel spectrogram
                    mel_spec = librosa.feature.melspectrogram(y=audio, sr=_, n_mels=64)
                    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

                    #mel_spec_db_fixed = librosa.util.fix_length(mel_spec_db, 9600, mode='constant', constant_values=0)

                    mel_spec_db_fixed = librosa.util.fix_length(mel_spec_db, size=960)

                    X.append(mel_spec_db_fixed)
                    Y.append(label)

#print(f"Shape of X before flattening: {X.shape}")

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)
#X = np.array([x.flatten() for x in X])
#Y = np.array(Y)


# Expand dimensions for Conv2D input
X = np.expand_dims(X, axis=-1)

# Convert labels to categorical if using categorical_crossentropy as the loss
# y = to_categorical(y)


In [7]:
# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
'''model = Sequential()

# First convolutional block
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

# Second convolutional block
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

# Flatten and feed into dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation="softmax"))

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])'''

In [8]:
model = Sequential()

# First convolutional block
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.BatchNormalization())

# Second convolutional block
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.BatchNormalization())

# Flatten and feed into dense layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation="softmax"))

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy", "precision", "recall", "f1_score"])

# Train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))


Epoch 1/10
70/70 [==============================] - 700s 10s/step - loss: 3.3297 - accuracy: 0.5192 - val_loss: 1.3066 - val_accuracy: 0.5893
Epoch 2/10
70/70 [==============================] - 735s 11s/step - loss: 0.4351 - accuracy: 0.8290 - val_loss: 1.1367 - val_accuracy: 0.8054
Epoch 3/10
70/70 [==============================] - 734s 11s/step - loss: 0.2807 - accuracy: 0.8862 - val_loss: 0.6687 - val_accuracy: 0.9286
Epoch 4/10
70/70 [==============================] - 730s 10s/step - loss: 0.1931 - accuracy: 0.9201 - val_loss: 0.2670 - val_accuracy: 0.9857
Epoch 5/10
70/70 [==============================] - 690s 10s/step - loss: 0.1380 - accuracy: 0.9406 - val_loss: 0.1519 - val_accuracy: 0.9821
Epoch 6/10
70/70 [==============================] - 733s 10s/step - loss: 0.1110 - accuracy: 0.9531 - val_loss: 0.0399 - val_accuracy: 0.9929
Epoch 7/10
70/70 [==============================] - 727s 10s/step - loss: 0.0967 - accuracy: 0.9607 - val_loss: 0.0205 - val_accuracy: 0.9946
Epoch 

In [11]:
# Plot training history
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend(loc='lower right')
plt.show()


NameError: name 'history' is not defined

In [20]:
# Train the model in smaller batches
'''batch_size = 32
for epoch in range(100):
    for start in range(0, len(X_train), batch_size):
        end = start + batch_size
        model.train_on_batch(X_train[start:end], Y_train[start:end])'''

In [ ]:
#model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, _test))


In [9]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

18/18 [==============================] - 42s 2s/step - loss: 0.0228 - accuracy: 0.9929
Loss: 0.022754773497581482
Accuracy: 0.9928571581840515
